# **Question 2 (b)**

### Imports

You should be able to complete the entire assignment using only the following imports. Please consult the course staff if you are unsure about whether additional packages may be used.

In [7]:
## Import Packages
import random
import numpy as np
import matplotlib.pyplot as plt

In [11]:
#from os import O_SEQUENTIAL
class Value:

    """
    Basic unit of storing a single scalar value and its gradient
    """

    def __init__(self, data, _children=()):
        """

        """
        self.data = data
        self.grad = 0
        self._prev = set(_children)
        self._backward = lambda: None

    def __add__(self, other):
        """
        Example implementation of a single class operation (addition)

        Args:
            other (Any): Node to add with the class

        Returns:
            out (callable): Function to referesh the gradient
        """
        #Firstly, convert some default value type in python to Value
        #Then do operations with two or more Value object
        other = other if isinstance(other, Value) else Value(other)

        #Secondly, create a new Value object which is the result of the operation
        out = Value(self.data + other.data, (self, other))

        #Thirdly, create a _backward function for the output object to refresh
        # the gradient of its _childrens,
        #Then assign this _backward function to the output object.
        def _backward():
            self.grad += out.grad * 1.0
            other.grad += out.grad * 1.0
        out._backward = _backward

        return out

    def __mul__(self, other):
        """
        Multiplication operation (e.g. Value(3) * Value(2) = Value(6))
        """
        #Convert default value type to Value
        other = other if isinstance(other, Value) else Value(other)

        #Create a new Value object with multiplied value
        out = Value(self.data * other.data, (self, other))

        #_backward function will pass the coefficient times the gradient from children nodes
        def _backward():
            self.grad += out.grad * other.data
            other.grad += out.grad * self.data
        out._backward = _backward

        return out

    def __pow__(self, other):
        """
        Power operation (e.g Value(3) ** 2 = Value(9))
        """
        #TODO implement power operation, we don't need to convert the exponent to Value
        assert isinstance(other, (int, float))

        #Create a new Value object with multiplied value
        out = Value(pow(self.data, other), {self})

        #_backward function will pass polynomial derivative from children node
        def _backward():
          if other == 1:
            self.grad = out.grad
          elif other == 0:
            self.grad = 0
          else:
            self.grad += out.grad * (other * pow(self.data, other-1))
        out._backward = _backward

        return out

    def relu(self):
        """
        ReLU activation function applied to the current Value
        """
        #TODO implement the relu activation function for the value itself.
        out = Value(max(self.data, 0), (self,))

        def _backward():
            self.grad += out.grad if self.data > 0 else 0
        out._backward = _backward

        return out


    def exp(self):
        """
        Exponentiate the current Value (e.g. e ^ Value(0) = Value(1))
        """
        #TODO implement the exponential function for and treat the value as exponent.
        #The base is natural e, you can use numpy to calculate the value of the exponential.

        #Create a new Value object with exp-ed value
        #float128 to avoid overflow
        exp_val = np.exp(np.float128(self.data))
        out = Value(exp_val, (self,))

        #_backward function will pass exp derivative to children node
        def _backward():
          self.grad += out.grad * exp_val
        out._backward = _backward

        return out

    def log(self):
        """
        Take the natural logarithm (base e) of the current Value
        """
        #TODO implement the logarithm function for and treat the value as exponent.
        #The bottom number should be e, you can use numpy to calculate the value of the logarithm.

        #Create a new Value object with log-ed value
        out = Value(np.log(self.data), {self})

        #_backward function will pass log derivative to children node
        def _backward():
          self.grad += out.grad * (1/self.data)
        out._backward = _backward

        return out

    def sigmoid(self):
        out = Value(1/(1+(np.exp(-self.data))), {self})

        #_backward function will pass log derivative to children node
        def _backward():
          self.grad += out.grad * (out.data) * (1-out.data)
        out._backward = _backward

        return out


    def topoSort(self, topo, visited):
      for nodes in self._prev:
        if not nodes in visited:
          nodes.topoSort(topo, visited)
      topo.insert(0, self)
      visited.add(self)



    def backward(self):
        """
        Run backpropagation from the current Value
        """
        #This function is called when you start backpropagation from this Value

        #The gradient of this value is initialized to 1 for you.
        self.grad = 1

        #You need to find a right topological order all of the children in the graph.
        #As for topology sort, you can refer to http://www.cs.cornell.edu/courses/cs312/2004fa/lectures/lecture15.htm
        """ using stack is computationally expansive, making the runtime increased severely
        def topoSort(value, visited, stack):
          visited.append(value)
          for node in value._prev:
            if node not in visited:
              topoSort(node, visited, stack)
          stack.append(0, value)

        def topoList(value, topo):
          stack = []
          visited = []
          topoSort(value, visited, stack)
          while(len(stack) != 0):
            v = stack.pop()
            topo.append(v)
        """

        topo = []
        visited = []
        #TODO find the right list of Value to be traversed
        '''
        Hint: you can recursively visit all non-visited node from the node calling backward.
        add one node to the head of the list after all of its children node are visited
        '''
        self.topoSort(topo, set())

        #topoSort(self, visited, topo)

        #go one variable at a time and apply the chain rule to get its gradient
        for v in topo:
            v._backward()

    # We handled the negation and reverse operations for you
    def __neg__(self): # -self
        """
        Negate the current Value
        """
        return self * -1

    def __radd__(self, other): #other + self
        """
        Reverse addition operation (ordering matters in Python)
        """
        return self + other

    def __sub__(self, other): # self - other
        """
        Subtraction operation
        """
        return self + (-other)

    def __rsub__(self, other): # other - self
        """
        Reverse subtraction operation
        """
        return other + (-self)

    def __rmul__(self, other): # other * self
        """
        Reverse multiplication operation
        """
        return self * other

    def __truediv__(self, other): # self / other
        """
        Division operation
        """
        return self * other**-1

    def __rtruediv__(self, other): # other / self
        """
        Reverse diction operation
        """
        return other * self**-1

    def __repr__(self):
        """
        Class representation (instead of unfriendly memory address)
        """
        return f"Value(data={self.data}, grad={self.grad})"

    # extra function to make Value object comparable for finding the max
    def __gt__(self, other):
        return self.data > other.data

In [12]:
nodes = []
names = []
## Initialize Example Values (From Written Assignment)
x = Value(0.63)
w1 = Value(0.25)
w2 = Value(-0.11)
w3 = Value(0.78)

# Initialize the computational graph
multiply1 = w1.__mul__(x)
sig1 = multiply1.sigmoid()
multiply2 = w2.__mul__(sig1)
sig2 = multiply2.sigmoid()
multiply3 = w3.__mul__(sig2)
sig3 = multiply3.sigmoid()
loss = -sig3.log()
nodes.append(multiply1)
nodes.append(multiply2)
nodes.append(multiply3)
nodes.append(sig1)
nodes.append(sig2)
nodes.append(sig3)
nodes.append(loss)
names.append("multiply1")
names.append("multiply2")
names.append("multiply3")
names.append("sig1")
names.append("sig2")
names.append("sig3")
names.append("loss")

# Calculate back propagation
loss.backward()


#TODO
#Do calculation for the question 1.b, and call backward to start backpropagation.
#Then print out the gradient of w1 w2 x1 x2.
for i in range(len(nodes)):
  print(names[i], nodes[i])

print("w1",w1)
print("w2",w2)
print("w3",w3)
print("x",x)


multiply1 Value(data=0.1575, grad=0.00216451138525994)
multiply2 Value(data=-0.059322318648819546, grad=-0.07919863670687821)
multiply3 Value(data=0.37843553907524874, grad=-0.4065042813639045)
sig1 Value(data=0.5392938058983595, grad=0.008711850037756604)
sig2 Value(data=0.4851737680451907, grad=-0.3170733394638455)
sig3 Value(data=0.5934957186360955, grad=-1.6849321209899988)
loss Value(data=0.5217252787139592, grad=1)
w1 Value(data=0.25, grad=0.001363642172713762)
w2 Value(data=-0.11, grad=-0.04271133421161387)
w3 Value(data=0.78, grad=-0.19722521391582792)
x Value(data=0.63, grad=0.000541127846314985)


In [13]:
from numpy.lib.function_base import add_newdoc_ufunc
nodes = []
names = []
## Initialize Example Values (From Written Assignment)
x = Value(0.63)
w1 = Value(0.25)
w2 = Value(-0.11)
w3 = Value(0.78)

# Initialize the computational graph
multiply1 = w1.__mul__(x)
sig1 = multiply1.sigmoid()
multiply2 = w2.__mul__(sig1)
sig2 = multiply2.sigmoid()
multiply3 = w3.__mul__(sig2)
add = multiply3.__add__(-128)
loss = add.__pow__(2)
nodes.append(multiply1)
nodes.append(multiply2)
nodes.append(multiply3)
nodes.append(sig1)
nodes.append(sig2)

nodes.append(loss)
names.append("multiply1")
names.append("multiply2")
names.append("multiply3")
names.append("sig1")
names.append("sig2")

names.append("loss")

# Calculate back propagation
loss.backward()


#TODO
#Do calculation for the question 1.b, and call backward to start backpropagation.
#Then print out the gradient of w1 w2 x1 x2.
for i in range(len(nodes)):
  print(names[i], nodes[i])

print("w1",w1)
print("w2",w2)
print("w3",w3)
print("x",x)


multiply1 Value(data=0.1575, grad=1.35909185681155)
multiply2 Value(data=-0.059322318648819546, grad=-49.7286468215865)
multiply3 Value(data=0.37843553907524874, grad=-255.2431289218495)
sig1 Value(data=0.5392938058983595, grad=5.470151150374515)
sig2 Value(data=0.4851737680451907, grad=-199.0896405590426)
loss Value(data=16287.26371545397, grad=1)
w1 Value(data=0.25, grad=0.8562278697912765)
w2 Value(data=-0.11, grad=-26.818351206588744)
w3 Value(data=0.78, grad=-123.83727062665811)
x Value(data=0.63, grad=0.3397729642028875)
